In [2]:
import pandas as pd
import DB as db
import numpy as np

In [ ]:
log_summary_file_name = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2022-12-11.zip"
log_summary_file_name = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2023-04-20.zip"
# log_summary_file_name = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2023-04-19.zip"
# log_summary_file_name = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2023-04-18.zip"

logdf= pd.read_csv(log_summary_file_name, low_memory=False).query("token == 'confirmLandReservation True'")\
    .drop(columns=['node', 'task_id','project_id', 'error_line'])
# logdf.head()

In [ ]:
# logdf.query("Land_ID in (80312, 80487)")
logdf.query("NID == '26312132702012'")

In [ ]:
land_app_fn = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\DB-summary\DB checksum\NewQueryLand.zip"
land_app_fn =r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\checksum\NewQueryLand.zip"
dbdf= pd.read_csv(land_app_fn, low_memory=False, dtype={'applicant_national_id':str}).query("project_id == 86")
    # .query("project_id == 86 and land_application_status == 'W'")[['applicant_national_id', 'check_sum', 'land_id', 'project_id']]

In [ ]:
dbdf.query("land_id == '80487'")

In [ ]:
df = (dbdf.merge(logdf, left_on='applicant_national_id', right_on='NID')
      .assign(match = lambda x:(x.checksum == x.check_sum))
    #   .fillna('land_id', axis=1)
    #   .assign(land_id = lambda x: x.astype(np.int64) )
    )
# df.info()

In [ ]:
df.match.value_counts()

In [ ]:
df.query("match == False").assign(land_id = lambda x: x.land_id.astype(np.int64)).to_excel(r"C:\Users\yahia\Downloads\mismatch.xlsx")

## UNITS

In [10]:
log_summary_file_name = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2023-02-07.zip"

logdf= pd.read_csv(log_summary_file_name, low_memory=False, dtype={'NID':str}).query("token == 'confirmReservation True'")\
    .drop(columns=['node', 'task_id','project_id', 'error_line'])



In [11]:
logdf.query("NID == '26411200102736'")

,log_date,line_no,NID,log_type,country,IP_address,service,token,categ,Gov,City,Region,District,Sub_District,Floor_No,building_no,Unit_No,Unit_Model,Unit_ID,checksum
42650,2023-02-07 10:32:58,6012965,26411200102736,ERROR,EG,197.162.113.115,confirmReservation,confirmReservation True,user,القاهرة,القاهرة الجديدة,نزهة الأندلس عمارات مدخل (أ),NaN,NaN,0.0,15.0,17.0,-,157427.0,0141997DCF07912E00BFFCD7637391BB3E6EDC64E6D1FB...


In [ ]:
logdf.token.value_counts()
logdf.to_csv('./out/mis_log.csv')

In [4]:
unit_app_fn = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\checksum\checksum.zip"
dbdf= pd.read_csv(unit_app_fn, low_memory=False, dtype={'applicant_national_id':str}).query("project_id == 87")

In [5]:
dbdf.query("applicant_national_id == '26411200102736'")

,project_id,applicant_national_id,check_sum,land_id
37689,87,26411200102736,0141997DCF07912E00BFFCD7637391BB3E6EDC64E6D1FB...,157427


In [ ]:
dbdf.project_id.value_counts().sort_index(ascending=False)

In [ ]:
dbdf.any()

In [ ]:
df = (dbdf.merge(logdf, how='outer', left_on='applicant_national_id', right_on='NID')
      # .assign(match = lambda x:(x.checksum == x.check_sum))
)


In [ ]:
df.to_csv('./out/checksum.csv')

In [ ]:
df.match.value_counts()

In [ ]:
df.query("match == False").assign(unit_id = lambda x: x.unit_id.astype(np.int64))[['NID', 'check_sum', 'checksum']]
#.to_excel(r"C:\Users\yahia\Downloads\mismatch.xlsx")

In [23]:
import json
log = """{"datetime":"2022-12-25T10:00:29.863","nid":28809142602376, "Country":"EG","Address":"156.192.68.49, 156.192.68.49", "service":"confirmLandReservation", "success":true,"details":{"Gov":"اسيوط","City":"أسيوط الجديدة","Region":"الحي الثاني","District":"الحي الثاني","Sub_District":"-","Land_No":"9","land_size":"475","excellence_ratio":"1.130","checksum":"0103DC22A4DB25EF9FA81B39D8CED893F0A0411335A2FB2D29A3BE87E95FFA1584"}}"""
res = json.loads(log)
res
# d = (res.get('details'))
det_lst = list(res.get('details').values())
if not res.get('details').get('Land_ID'):
    det_lst.insert(1,None)



['اسيوط',
 None,
 'أسيوط الجديدة',
 'الحي الثاني',
 'الحي الثاني',
 '-',
 '9',
 '475',
 '1.130',
 '0103DC22A4DB25EF9FA81B39D8CED893F0A0411335A2FB2D29A3BE87E95FFA1584']